<a href="https://colab.research.google.com/github/chethanmo/chethanmo.github.io/blob/master/NEW_teststrat.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install pandas matplotlib yfinance

In [ ]:
!curl -L http://prdownloads.sourceforge.net/ta-lib/ta-lib-0.4.0-src.tar.gz -O && tar xzvf ta-lib-0.4.0-src.tar.gz
!cd ta-lib && ./configure --prefix=/usr && make && make install && cd - && pip install ta-lib

In [14]:
import pandas as pd
import talib as ta
import yfinance as yf
from datetime import date as date
from dateutil.relativedelta import relativedelta
import matplotlib.pyplot as plt

In [ ]:
file_path = '/content/EQUITY_L.csv'
fields = ['SYMBOL']
tickers = pd.read_csv(file_path, usecols=fields)

ath_list = []
for i in range(0,len(tickers)):
  ath_list.append(tickers['SYMBOL'][i] +'.NS')

unwanted = {'INDO-RE.NS','CDSL.NS','HERIT-RE.NS','ZODIAC.NS','EMAMIPAP.NS','DIGJAMLMTD.NS','GUJRAFFIA.NS','NIRAJISPAT.NS','KALYANI.NS','ARTEMISMED.NS'}
ath_list = [e for e in ath_list if e not in unwanted]

# ath_list

In [21]:
startdate = date.today() - relativedelta(months=1)
enddate = date.today() + relativedelta(days=1)

# x = ticker_list['Ticker'].to_string(header=False,index=False).split('\n')
# tickers = [','.join(ele.split()) for ele in x]
try:
  df_1d = yf.download(ath_list, start= startdate, end= enddate)
except:
  pass

df_1d

[*********************100%***********************]  1862 of 1862 completed


Adj Close                                                        \
           20MICRONS.NS 21STCENMGM.NS    360ONE.NS 3IINFOLTD.NS    3MINDIA.NS   
Date                                                                            
2023-02-01    78.900002     21.000000  1891.949951    36.950001  22645.449219   
2023-02-02    79.800003     20.600000  1886.599976    37.099998  23140.500000   
2023-02-03    78.000000     20.350000  1899.250000    36.299999  23033.449219   
2023-02-06    77.849998     20.549999  1861.699951    38.000000  22612.400391   
2023-02-07    76.400002     20.549999  1839.900024    37.349998  22900.050781   
2023-02-08    76.949997     20.600000  1838.349976    37.150002  23117.800781   
2023-02-09    77.349998     20.750000  1824.650024    36.950001  23528.349609   
2023-02-10    76.550003     20.350000  1856.949951    36.700001  23420.349609   
2023-02-13    76.550003     19.950001  1852.699951    36.099998  22956.949219   
2023-02-14    75.250000     19.549999  1837.099976    35.549999  22376.349609   
2023-02-15    75.550003     19.150000  1839.000000    35.700001  21554.000000   
2023-02-16    73.650002     18.799999  1822.699951    36.450001  22263.500000   
2023-02-17    72.050003     18.450001  1813.300049    36.250000  22177.050781   
2023-02-20    70.050003     18.100000  1831.250000    35.799999  22085.050781   
2023-02-21    70.199997     17.750000  1812.949951    35.650002  22212.349609   
2023-02-22    68.550003     17.400000  1785.250000    35.250000  21809.000000   
2023-02-23    69.150002     17.049999  1785.550049    34.650002  22091.150391   
2023-02-24          NaN     17.350000  1776.699951          NaN           NaN   
2023-02-27    67.150002     17.549999  1779.750000    32.700001  21944.500000   
2023-02-28    67.300003     17.350000  1723.449951    32.049999  22888.849609   
2023-03-01    70.000000     17.000000  1775.000000    33.349998  22600.000000   

                                                                      ...  \
            3PLAND.NS  4THDIM.NS   5PAISA.NS  63MOONS.NS A2ZINFRA.NS  ...   
Date                                                                  ...   
2023-02-01  21.299999  27.950001  291.100006  166.500000        8.50  ...   
2023-02-02  20.250000  28.150000  287.799988  170.800003        8.55  ...   
2023-02-03  19.250000  28.549999  287.299988  180.600006        8.30  ...   
2023-02-06  18.400000  29.100000  293.549988  185.100006        8.15  ...   
2023-02-07  18.600000  29.500000  294.100006  177.600006        8.00  ...   
2023-02-08  18.250000  29.900000  302.000000  194.649994        8.15  ...   
2023-02-09  19.150000  30.350000  295.549988  192.850006        8.20  ...   
2023-02-10  20.100000  30.799999  294.899994  198.699997        8.40  ...   
2023-02-13  21.100000  31.350000  292.399994  207.500000        8.45  ...   
2023-02-14  22.150000  31.950001  285.799988  213.100006        8.10  ...   
2023-02-15  23.250000  32.549999  284.850006  212.699997        7.90  ...   
2023-02-16  24.400000  33.049999  291.000000  206.750000        8.10  ...   
2023-02-17  23.200001  33.650002  291.350006  203.949997        8.00  ...   
2023-02-20  22.049999  34.299999  288.200012  203.250000        7.90  ...   
2023-02-21  21.350000  34.799999  285.600006  197.050003        7.90  ...   
2023-02-22  20.350000  35.049999  286.200012  193.550003        7.60  ...   
2023-02-23  20.049999  35.700001  285.899994  194.699997        7.45  ...   
2023-02-24  21.049999        NaN         NaN  190.399994        7.30  ...   
2023-02-27  22.100000  37.000000  272.250000  197.000000        6.95  ...   
2023-02-28  22.950001  37.549999  274.350006  204.449997        7.00  ...   
2023-03-01  21.900000  37.900002  277.350006  201.649994        7.30  ...   

              Volume                                                          \
           ZENTEC.NS ZFCVINDIA.NS ZIMLAB.NS ZODIACLOTH.NS  ZOMATO.NS ZOTA.NS   
Date                                                        

In [32]:
df_1d = df_1d.swaplevel(0, 1, 1).sort_index(axis=1)

In [33]:
df_1d

20MICRONS.NS                                                        \
              Adj Close      Close       High        Low       Open    Volume   
Date                                                                            
2023-02-01    78.900002  78.900002  82.449997  78.400002  81.400002   95985.0   
2023-02-02    79.800003  79.800003  81.500000  77.599998  77.800003  105800.0   
2023-02-03    78.000000  78.000000  80.800003  77.599998  80.400002   78094.0   
2023-02-06    77.849998  77.849998  79.150002  76.199997  79.150002   68209.0   
2023-02-07    76.400002  76.400002  79.349998  76.000000  78.000000   86932.0   
2023-02-08    76.949997  76.949997  77.949997  76.400002  76.849998   73079.0   
2023-02-09    77.349998  77.349998  77.699997  76.250000  76.949997   38410.0   
2023-02-10    76.550003  76.550003  77.900002  76.349998  77.349998   50165.0   
2023-02-13    76.550003  76.550003  77.500000  75.599998  77.500000   45846.0   
2023-02-14    75.250000  75.250000  76.900002  74.500000  76.500000   68154.0   
2023-02-15    75.550003  75.550003  76.650002  74.699997  74.699997   83127.0   
2023-02-16    73.650002  73.650002  77.000000  73.199997  75.949997  157289.0   
2023-02-17    72.050003  72.050003  74.349998  71.900002  73.699997   89627.0   
2023-02-20    70.050003  70.050003  73.000000  69.000000  72.099998  116487.0   
2023-02-21    70.199997  70.199997  71.250000  69.500000  70.000000   82498.0   
2023-02-22    68.550003  68.550003  71.000000  67.599998  70.199997  117807.0   
2023-02-23    69.150002  69.150002  70.099998  68.300003  68.849998   59288.0   
2023-02-24          NaN        NaN        NaN        NaN        NaN       NaN   
2023-02-27    67.150002  67.150002  69.150002  66.800003  68.300003   48886.0   
2023-02-28    67.300003  67.300003  68.250000  66.650002  67.099998   34163.0   
2023-03-01    70.000000  70.000000  71.099998  67.050003  67.150002  172748.0   

           21STCENMGM.NS                                   ... ZYDUSLIFE.NS  \
               Adj Close      Close       High        Low  ...         High   
Date                                                       ...                
2023-02-01     21.000000  21.000000  21.400000  20.600000  ...   444.500000   
2023-02-02     20.600000  20.600000  21.200001  20.600000  ...   440.950012   
2023-02-03     20.350000  20.350000  20.799999  20.200001  ...   442.000000   
2023-02-06     20.549999  20.549999  20.750000  20.049999  ...   476.500000   
2023-02-07     20.549999  20.549999  20.950001  20.250000  ...   480.700012   
2023-02-08     20.600000  20.600000  20.850000  20.200001  ...   482.250000   
2023-02-09     20.750000  20.750000  20.950001  20.299999  ...   480.500000   
2023-02-10     20.350000  20.350000  20.850000  20.350000  ...   477.450012   
2023-02-13     19.950001  19.950001  20.650000  19.950001  ...   476.850006   
2023-02-14     19.549999  19.549999  20.299999  19.549999  ...   474.899994   
2023-02-15     19.150000  19.150000  19.150000  19.150000  ...   474.700012   
2023-02-16     18.799999  18.799999  19.000000  18.799999  ...   480.000000   
2023-02-17     18.450001  18.450001  18.450001  18.450001  ...   479.149994   
2023-02-20     18.100000  18.100000  18.799999  18.100000  ...   473.350006   
2023-02-21     17.750000  17.750000  18.100000  17.750000  ...   472.299988   
2023-02-22     17.400000  17.400000  18.100000  17.400000  ...   469.450012   
2023-02-23     17.049999  17.049999  17.700001  17.049999  ...   469.950012   
2023-02-24     17.350000  17.350000  17.350000  16.700001  ...          NaN   
2023-02-27     17.549999  17.549999  17.650000  17.000000  ...   466.350006   
2023-02-28     17.350000  17.350000  17.900000  17.200001  ...   467.000000   
2023-03-01     17.000000  17.000000  17.500000  17.000000  ...   476.000000   

                                              ZYDUSWELL.NS               \
                   Low        Open     Volume    Adj Close        Close   
Date               

In [30]:
df_open = pd.DataFrame
df_high = pd.DataFrame
df_low = pd.DataFrame
df_close = pd.DataFrame

df_open = df_1d.loc[:,'Open']
df_high = df_1d.loc[:,'High']
df_low = df_1d.loc[:,'Low']
df_close = df_1d.loc[:,'Close']

# result = ta.CDLENGULFING(df_1d[:,'Open'], df_1d[:,'High'], df_1d[:,'Low'], df_1d[:,'Close'] )

In [34]:
df_open 

,20MICRONS.NS,21STCENMGM.NS,360ONE.NS,3IINFOLTD.NS,3MINDIA.NS,3PLAND.NS,4THDIM.NS,5PAISA.NS,63MOONS.NS,A2ZINFRA.NS,...,ZENTEC.NS,ZFCVINDIA.NS,ZIMLAB.NS,ZODIACLOTH.NS,ZOMATO.NS,ZOTA.NS,ZUARI.NS,ZUARIIND.NS,ZYDUSLIFE.NS,ZYDUSWELL.NS
Date,,,,,,,,,,,,,,,,,,,,,
2023-02-01,81.400002,21.000000,1827.300049,38.349998,23329.050781,23.000000,28.150000,302.049988,166.899994,9.10,...,204.500000,9295.000000,84.449997,96.050003,50.000000,347.950012,146.600006,134.600006,434.100006,1405.550049
2023-02-02,77.800003,21.200001,1888.050049,36.450001,22696.000000,21.299999,28.049999,290.000000,164.100006,8.40,...,193.399994,9410.000000,85.550003,92.000000,48.400002,320.500000,150.399994,130.149994,434.000000,1423.000000
2023-02-03,80.400002,20.799999,1891.300049,37.549999,23149.000000,20.250000,28.549999,298.700012,170.800003,8.75,...,198.899994,9497.000000,80.300003,92.400002,48.150002,324.000000,152.300003,134.500000,433.950012,1405.000000
2023-02-06,79.150002,20.049999,1907.000000,36.000000,23100.000000,19.950001,28.850000,287.299988,195.000000,8.10,...,199.949997,9590.150391,85.800003,93.900002,48.700001,310.100006,151.899994,129.899994,452.000000,1397.050049
2023-02-07,78.000000,20.549999,1861.050049,38.200001,22677.949219,18.000000,29.299999,297.000000,185.100006,8.55,...,198.000000,9665.000000,82.050003,96.300003,48.000000,307.950012,155.000000,131.899994,470.200012,1469.000000
2023-02-08,76.849998,20.250000,1815.050049,37.650002,22900.050781,18.900000,29.950001,294.950012,179.350006,8.10,...,202.699997,9838.000000,81.349998,96.199997,51.200001,310.950012,151.000000,130.500000,480.149994,1440.000000
2023-02-09,76.949997,20.600000,1841.000000,37.200001,23250.000000,18.500000,30.250000,305.049988,194.050003,8.40,...,205.949997,9945.000000,82.949997,97.050003,54.700001,314.950012,154.199997,131.000000,469.700012,1438.199951
2023-02-10,77.349998,20.350000,1821.000000,36.900002,23630.000000,20.100000,30.600000,296.000000,192.350006,8.20,...,203.000000,9851.000000,80.699997,96.750000,52.700001,315.899994,150.399994,128.500000,473.500000,1410.000000
2023-02-13,77.500000,19.950001,1856.949951,36.849998,23310.000000,21.100000,31.000000,300.000000,199.899994,8.65,...,213.800003,9992.000000,81.349998,94.900002,53.000000,311.000000,152.699997,129.000000,476.850006,1415.000000


In [31]:
result = ta.CDLENGULFING(df_open, df_high, df_low, df_close )

TypeError: ignored